This notebook is unstable scratch paper for my ongoing experiments.

In [ ]:
#Setup

import gym
import sys
sys.path.append('./src')
import torch
device = torch.device("cuda")

env_id = "ALE/Pong-v5"
#fbfile = "pong.dat"
fbfile = "pong_synth.dat"
#env_id = 'CartPole-v1'
#fbfile = 'testcartpole.dat'
#env_id = 'LunarLander-v2'
#fbfile = "lunarlandersynth.dat"
#fbfile = "lunarlandernormal.dat"
#fbfile = "lunarlander2.dat"
#env_id = "BipedalWalker-v3"
#fbfile = "bipedal1.dat"
# Create the env
env = gym.make(env_id)

env.reset()


from feedback import FeedbackManager

fb = FeedbackManager(fbfile, show_picker=True)

from rewardmodel import MlpRewardModel, ConvRewardModel
from rewardensemble import RewardEnsemble

def makemodelfun():
    def _f():
        #return MlpRewardModel(env.observation_space.shape, device)
        return ConvRewardModel(env.observation_space.shape, device)
    return _f

#rewardmodel = MlpRewardModel(env.observation_space.shape, device)
rewardensemble = RewardEnsemble(3, makemodelfun())

from envwithrewardmodel import EnvWithRewardModel
def makeenvfun():
    def _f():
        #env_id specified higher in the notebook
        base_env = gym.make(env_id)
        return EnvWithRewardModel(base_env, rewardensemble)
    return _f



from stable_baselines3.common.vec_env import DummyVecEnv
vec_env = DummyVecEnv([makeenvfun() for i in range(4)])

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C



#model = A2C("MlpPolicy", vec_env, verbose=1)
model = A2C("CnnPolicy", vec_env, verbose=1)

#Train reward model
def train_reward(n):
    for i in range(n):
        #batch = fb.randomBatch(20)
        #rewardmodel.train(batch)
        batches = fb.batchForEnsemble(20, rewardensemble.n)
        rewardensemble.train(batches)
        
        
#Train agent
def train_a2c(n):
    for i in range(n):
        model.learn(total_timesteps=1000, reset_num_timesteps=False)

        mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
        print(f"{(i+1) * 1000}: mean {mean_reward}, std {std_reward}")
        
from harvestclip import harvestClips, harvestSynthetic, harvestWithEnsemble

harvest_env = gym.make(env_id)
def harvest(n):
    for i in range(n):
        clips, obs = harvestClips(harvest_env, model, n_timesteps=300)
        fb.queueClips(clips[0], clips[1], obs[0], obs[1])
        
def harvest_ensemble(n):
    for i in range(n):
        clips, obs = harvestWithEnsemble(harvest_env, model, rewardensemble, n_timesteps=300)
        fb.queueClips(clips[0], clips[1], obs[0], obs[1])
        
def add_synth_feedback(n):
    for i in range(n):
        fb.addComparison(harvestSynthetic(harvest_env, model, n_timesteps=300))


The bits of the training loop. Use them in any order as needed.

In [ ]:
add_synth_feedback(200)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(fb.comparisons[0]["observations"][0][0])

In [ ]:
train_reward(10)

In [ ]:
train_a2c(10)

In [ ]:
harvest(20)

In [ ]:
harvest_ensemble(10)

Below here we have some situational stuff that you can use out of order.

In [ ]:
fb.save()

In [ ]:
fb.viewComparisons()

In [ ]:
from envplayer import EnvPlayer

player_env = gym.make(env_id)
player = EnvPlayer(player_env, model, rewardensemble)

In [ ]:
#model.save("a2c_lunarlander")

In [ ]:
evals = rewardensemble.models[0].naneval
evals

In [ ]:
evals.shape